In [2]:
import warnings
warnings.filterwarnings('ignore')
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func, Table, Column, Integer, String, MetaData
from sqlite3 import Error
import sqlite3

In [3]:
from pokemontcgsdk import Card
from pokemontcgsdk import Set
from pokemontcgsdk import Type
from pokemontcgsdk import Supertype
from pokemontcgsdk import Subtype


In [4]:
import pandas as pd
import numpy as np

In [5]:
# Retreive Card Data (This will take a while)
cards = Card.all()

In [6]:
#Pull card information into lists.

x = 0
card_name = []
card_id = []
national_pokedex_number = []
image_url = []
image_url_hi_res = []
card_subtype = []
card_ability = []
card_ancient_trait = []
card_hp = []
card_number = []
card_artist = []
card_rarity = []
card_series = []
card_set = []
card_set_code = []
card_converted_retreat_cost = []
card_text = []
card_types = []
card_attacks = []
card_weaknesses = []
card_resistances = []
card_evolves_from = []
for row in cards:
    card_id.append(row.id)
    card_name.append(row.name)
    national_pokedex_number.append(row.national_pokedex_number)
    image_url.append(row.image_url)
    image_url_hi_res.append(row.image_url_hi_res)
    card_subtype.append(row.subtype)
    card_ability.append(row.ability)
    card_ancient_trait.append(row.ancient_trait)
    card_hp.append(row.hp)
    card_number.append(row.number)
    card_artist.append(row.artist)
    card_rarity.append(row.rarity)
    card_series.append(row.series)
    card_set.append(row.set)
    card_set_code.append(row.set_code)
    card_converted_retreat_cost.append(row.converted_retreat_cost)
    card_text.append(row.text)
    card_types.append(row.types)
    card_attacks.append(row.attacks)
    card_weaknesses.append(row.weaknesses)
    card_resistances.append(row.resistances)
    card_evolves_from.append(row.evolves_from)
#     x = x + 1
#     if x == 5:
#         break

In [7]:
#Convert list to pandas DataFrame
pokemon_cards = pd.DataFrame({
    'card_name':card_name,
    'card_id':card_id,
    'national_pokedex_number':national_pokedex_number,
    'image_url':image_url,
    'image_url_hi_res':image_url_hi_res,
    'card_subtype':card_subtype,
    'card_ability':card_ability,
    'card_ancient_trait':card_ancient_trait,
    'card_hp':card_hp,
    'card_number':card_number,
    'card_artist':card_artist,
    'card_rarity':card_rarity,
    'card_series':card_series,
    'card_set':card_set,
    'card_set_code':card_set_code,
    'card_converted_retreat_cost':card_converted_retreat_cost,
    'card_text':card_text,
    'card_types':card_types,
    'card_attacks':card_attacks,
    'card_weaknesses':card_weaknesses,
    'card_resistances':card_resistances,
    'card_evolves_from':card_evolves_from
})

pokemon_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11901 entries, 0 to 11900
Data columns (total 22 columns):
card_name                      11901 non-null object
card_id                        11901 non-null object
national_pokedex_number        10078 non-null float64
image_url                      11901 non-null object
image_url_hi_res               11901 non-null object
card_subtype                   11896 non-null object
card_ability                   2393 non-null object
card_ancient_trait             39 non-null object
card_hp                        10380 non-null object
card_number                    11901 non-null object
card_artist                    11787 non-null object
card_rarity                    11639 non-null object
card_series                    11901 non-null object
card_set                       11901 non-null object
card_set_code                  11901 non-null object
card_converted_retreat_cost    9964 non-null float64
card_text                      2923 non-null o

In [8]:
#Drop any NaN values from ['national_pokedex_number']
pokemon_cards = pokemon_cards[np.isfinite(pokemon_cards['national_pokedex_number'])]

In [9]:
#Retreive a list of unique Pokedex ids
unique_pokedex_number = [] 
for i in national_pokedex_number: 
    if i not in unique_pokedex_number: 
        unique_pokedex_number.append(i) 
    if i == 'None':
        pass

# Remove the None Value
unique_pokedex_number.pop(1)
#Remove the last 7 Pokedex numbers (They don't exist in the bulbapedia data yet)
unique_pokedex_number = unique_pokedex_number[:len(unique_pokedex_number)-7]

In [10]:
import requests
import json

base_url = 'https://pokeapi.co/api/v2/'
bulbapedia = pd.DataFrame({'pokedex_id': unique_pokedex_number})
x = 0
# itterate through rows and find the bulbapedia stats for each pokemon by national pokedex id.
for index, row in bulbapedia.iterrows():
    poke_id = str(round(row['pokedex_id']))
#     x = x + 1
#     if x == 10:
#         break
    query_url = base_url + '/pokemon/' + poke_id + '/'
    response = requests.get(query_url)
    response_json = response.json()
    json_dump = json.dumps(response_json,indent=1)
    

# create a new data frame with bulbapedia entries.
    try:
        bulbapedia.loc[index, "sprite"] = response_json['sprites']['front_default']
        bulbapedia.loc[index, "pokemon_name"] = response_json["name"]
        bulbapedia.loc[index, "pokemon_weight"] = response_json['weight']
        bulbapedia.loc[index, "pokemon_height"] = response_json['height']
        bulbapedia.loc[index, "speed"] = response_json["stats"][0]['base_stat']
        bulbapedia.loc[index, "special_defense"] = response_json["stats"][1]['base_stat']
        bulbapedia.loc[index, "special_attack"] = response_json["stats"][2]['base_stat']
        bulbapedia.loc[index, "defense"] = response_json["stats"][3]['base_stat']
        bulbapedia.loc[index, "attack"] = response_json["stats"][4]['base_stat']
        bulbapedia.loc[index, "hp"] = response_json["stats"][5]['base_stat']
        bulbapedia.loc[index, "pokemon_type1"] = response_json["types"][0]['type']['name']
        bulbapedia.loc[index, "pokemon_type2"] = response_json["types"][1]['type']['name']
        

    except (KeyError, IndexError, ValueError, json.JSONDecodeError):
        print(f'{poke_id} Could not find one or more Element')

104 Could not find one or more Element
387 Could not find one or more Element
182 Could not find one or more Element
297 Could not find one or more Element
716 Could not find one or more Element
200 Could not find one or more Element
692 Could not find one or more Element
262 Could not find one or more Element
204 Could not find one or more Element
362 Could not find one or more Element
181 Could not find one or more Element
126 Could not find one or more Element
4 Could not find one or more Element
477 Could not find one or more Element
684 Could not find one or more Element
143 Could not find one or more Element
261 Could not find one or more Element
466 Could not find one or more Element
201 Could not find one or more Element
301 Could not find one or more Element
300 Could not find one or more Element
399 Could not find one or more Element
659 Could not find one or more Element
405 Could not find one or more Element
455 Could not find one or more Element
59 Could not find one or mo

372 Could not find one or more Element
129 Could not find one or more Element
512 Could not find one or more Element
674 Could not find one or more Element
237 Could not find one or more Element
420 Could not find one or more Element
61 Could not find one or more Element
55 Could not find one or more Element
190 Could not find one or more Element
113 Could not find one or more Element
495 Could not find one or more Element
762 Could not find one or more Element
501 Could not find one or more Element
321 Could not find one or more Element
99 Could not find one or more Element
615 Could not find one or more Element
489 Could not find one or more Element
503 Could not find one or more Element
535 Could not find one or more Element
60 Could not find one or more Element
172 Could not find one or more Element
575 Could not find one or more Element
577 Could not find one or more Element
403 Could not find one or more Element
523 Could not find one or more Element
254 Could not find one or mor

In [11]:
#Preview Data Frame

#Set Max Column Width to Store Sprites Properly
pd.set_option('display.max_colwidth', -1)

#Sort by Pokedex ID
bulbapedia = bulbapedia.sort_values('pokedex_id')


In [12]:
species_url = 'https://pokeapi.co/api/v2/pokemon-species/'
x = 0
# itterate through rows and find the bulbapedia stats for each pokemon by national pokedex id.
for index, row in bulbapedia.iterrows():
    poke_id = str(round(row['pokedex_id']))
    query_url = species_url + poke_id + '/'
    response = requests.get(query_url)
    response_json = response.json()
    json_dump = json.dumps(response_json,indent=1)
    

#create a new data frame with bulbapedia entries.
    try:
        bulbapedia.loc[index, "base_hapiness"] = response_json["base_happiness"]
        bulbapedia.loc[index, "color_group"] = response_json['color']['name']
        bulbapedia.loc[index, "egg_group"] = response_json['egg_groups'][0]['name']
        bulbapedia.loc[index, "generation"] = response_json["generation"]['name']
        bulbapedia.loc[index, "shape"] = response_json["shape"]['name']
        

    except (KeyError, IndexError, ValueError, json.JSONDecodeError):
        print(f'{poke_id} One or More Element Not Found')

In [13]:
bulbapedia = bulbapedia.rename(columns={'pokedex_id': 'national_pokedex_number'})

In [14]:
merged_data = pd.merge(pokemon_cards, bulbapedia,
                       how='left', on='national_pokedex_number')

In [15]:
merged_data

,card_name,card_id,national_pokedex_number,image_url,image_url_hi_res,card_subtype,card_ability,card_ancient_trait,card_hp,card_number,...,defense,attack,hp,pokemon_type1,pokemon_type2,base_hapiness,color_group,egg_group,generation,shape
0,Cubone,dp6-90,104.0,https://images.pokemontcg.io/dp6/90.png,https://images.pokemontcg.io/dp6/90_hires.png,Basic,None,None,60,90,...,95.0,50.0,50.0,ground,NaN,70.0,brown,monster,generation-i,upright
1,Turtwig,pop9-17,387.0,https://images.pokemontcg.io/pop9/17.png,https://images.pokemontcg.io/pop9/17_hires.png,Basic,None,None,60,17,...,64.0,68.0,55.0,grass,NaN,70.0,green,plant,generation-iv,quadruped
2,Bellossom,xy7-4,182.0,https://images.pokemontcg.io/xy7/4.png,https://images.pokemontcg.io/xy7/4_hires.png,Stage 2,None,None,120,4,...,95.0,80.0,75.0,grass,NaN,70.0,green,plant,generation-ii,humanoid
3,Hariyama ex,ex8-100,297.0,https://images.pokemontcg.io/ex8/100.png,https://images.pokemontcg.io/ex8/100_hires.png,EX,"{'name': 'Commanding Aura', 'text': 'As long as Hariyama ex is your Active Pokémon, your opponent can't play any Stadium cards from his or her hand.', 'type': 'Poké-Body'}",None,110,100,...,60.0,120.0,144.0,fighting,NaN,70.0,brown,humanshape,generation-iii,humanoid
4,Xerneas,xyp-XY05,716.0,https://images.pokemontcg.io/xyp/XY05.png,https://images.pokemontcg.io/xyp/XY05_hires.png,Basic,None,None,130,XY05,...,95.0,131.0,126.0,fairy,NaN,0.0,blue,no-eggs,generation-vi,quadruped
5,Aggron,ex16-1,306.0,https://images.pokemontcg.io/ex16/1.png,https://images.pokemontcg.io/ex16/1_hires.png,Stage 2,"{'name': 'Terraforming', 'text': 'Once during your turn (before your attack), you may look at the top 5 cards from your deck and put them back on top of your deck in any order. This power can't be used if Aggron is affected by a Special Condition.', 'type': 'Poké-Power'}",None,110,1,...,180.0,110.0,70.0,rock,steel,35.0,gray,monster,generation-iii,upright
6,Misdreavus,dp6-107,200.0,https://images.pokemontcg.io/dp6/107.png,https://images.pokemontcg.io/dp6/107_hires.png,Basic,None,None,60,107,...,60.0,60.0,60.0,ghost,NaN,35.0,gray,indeterminate,generation-ii,ball
7,Greninja,xy0-14,658.0,https://images.pokemontcg.io/xy0/14.png,https://images.pokemontcg.io/xy0/14_hires.png,Stage 2,None,None,140,14,...,67.0,95.0,72.0,dark,water,70.0,blue,water1,generation-vi,humanoid
8,Clauncher,xy0-15,692.0,https://images.pokemontcg.io/xy0/15.png,https://images.pokemontcg.io/xy0/15_hires.png,Basic,None,None,70,15,...,62.0,53.0,50.0,water,NaN,70.0,blue,water3,generation-vi,armor
9,Magneton,ex16-16,82.0,https://images.pokemontcg.io/ex16/16.png,https://images.pokemontcg.io/ex16/16_hires.png,Stage 1,"{'name': 'Magnetic Field', 'text': 'Once during your turn (before your attack), if you have basic Energy cards in your discard pile, you may discard any 1 card from your hand. Then search for up to 2 basic Energy cards from your discard pile, show them to your opponent, and put them into your hand. You can't return the card you first discarded to your hand in this way. This power can't be used if Magneton is affected by a Special Condition.', 'type': 'Poké-Power'}",None,70,16,...,95.0,60.0,50.0,steel,electric,70.0,gray,mineral,generation-i,heads


In [16]:
merged_data.columns

Index(['card_name', 'card_id', 'national_pokedex_number', 'image_url',
       'image_url_hi_res', 'card_subtype', 'card_ability',
       'card_ancient_trait', 'card_hp', 'card_number', 'card_artist',
       'card_rarity', 'card_series', 'card_set', 'card_set_code',
       'card_converted_retreat_cost', 'card_text', 'card_types',
       'card_attacks', 'card_weaknesses', 'card_resistances',
       'card_evolves_from', 'sprite', 'pokemon_name', 'pokemon_weight',
       'pokemon_height', 'speed', 'special_defense', 'special_attack',
       'defense', 'attack', 'hp', 'pokemon_type1', 'pokemon_type2',
       'base_hapiness', 'color_group', 'egg_group', 'generation', 'shape'],
      dtype='object')

In [17]:
import pymongo
conn = 'mongodb://localhost:27017/'
client = pymongo.MongoClient(conn)

x = []
for index, row in merged_data.iterrows():
    post = {'card_name': row['card_name'], 
             'card_id': row['card_id'], 
             'national_pokedex_number': row['national_pokedex_number'], 
             'image_url': row['image_url'],
             'image_url_hi_res': row['image_url_hi_res'], 
             'card_subtype': row['card_subtype'], 
             'card_ability': row['card_ability'], 
             'card_ancient_trait': row['card_ancient_trait'], 
             'card_hp': row['card_hp'], 
             'card_number': row['card_number'], 
             'card_artist': row['card_artist'], 
             'card_rarity': row['card_rarity'], 
             'card_series': row['card_series'], 
             'card_set': row['card_set'], 
             'card_set_code': row['card_set_code'], 
             'card_converted_retreat_cost': row['card_converted_retreat_cost'], 
             'card_text': row['card_text'], 
             'card_types': row['card_types'],
             'card_attacks': row['card_attacks'], 
             'card_weaknesses': row['card_weaknesses'], 
             'card_resistances': row['card_resistances'],
             'card_evolves_from': row['card_evolves_from'], 
             'sprite': row['sprite'], 
             'pokemon_name': row['pokemon_name'], 
             'pokemon_weight': row['pokemon_weight'], 
             'pokemon_height': row['pokemon_height'], 
             'speed': row['speed'], 
             'special_defense': row['special_defense'], 
             'special_attack': row['special_attack'], 
             'defense': row['defense'], 
             'attack': row['attack'], 
             'hp': row['hp'], 
             'pokemon_type1': row['pokemon_type1'], 
             'pokemon_type2': row['pokemon_type2'],
             'base_hapiness': row['base_hapiness'], 
             'color_group': row['color_group'], 
             'egg_group': row['egg_group'], 
             'generation': row['generation'], 
             'shape': row['shape']}
    
    client.pokemon_db.pokemon_cards.insert_one(post)



In [22]:
post

{'card_name': 'Dragonite-GX',
 'card_id': 'sm11-248',
 'national_pokedex_number': 149.0,
 'image_url': 'https://images.pokemontcg.io/sm11/248.png',
 'image_url_hi_res': 'https://images.pokemontcg.io/sm11/248_hires.png',
 'card_subtype': 'Stage 2',
 'card_ability': None,
 'card_ancient_trait': None,
 'card_hp': '250',
 'card_number': '248',
 'card_artist': None,
 'card_rarity': 'Rare Secret',
 'card_series': 'Sun & Moon',
 'card_set': 'Unified Minds',
 'card_set_code': 'sm11',
 'card_converted_retreat_cost': 2.0,
 'card_text': None,
 'card_types': ['Dragon'],
 'card_attacks': [{'cost': ['Lightning', 'Water', 'Colorless'],
   'name': 'Dragon Claw',
   'text': '',
   'damage': '130',
   'convertedEnergyCost': 3},
  {'cost': ['Colorless', 'Colorless', 'Colorless', 'Colorless', 'Colorless'],
   'name': 'Sky Judgment',
   'text': 'Discard 3 Energy from this Pokémon.',
   'damage': '270',
   'convertedEnergyCost': 5},
  {'cost': ['Colorless'],
   'name': 'Mach Delivery GX',
   'text': "You ma

In [19]:
merged_data.sort_values('national_pokedex_number')

,card_name,card_id,national_pokedex_number,image_url,image_url_hi_res,card_subtype,card_ability,card_ancient_trait,card_hp,card_number,...,defense,attack,hp,pokemon_type1,pokemon_type2,base_hapiness,color_group,egg_group,generation,shape
6314,Erika's Bulbasaur,gym2-39,1.0,https://images.pokemontcg.io/gym2/39.png,https://images.pokemontcg.io/gym2/39_hires.png,Basic,None,None,50,39,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped
6126,Bulbasaur,ex6-54,1.0,https://images.pokemontcg.io/ex6/54.png,https://images.pokemontcg.io/ex6/54_hires.png,Basic,None,None,50,54,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped
8537,Bulbasaur,smp-SM198,1.0,https://images.pokemontcg.io/smp/SM198.png,https://images.pokemontcg.io/smp/SM198_hires.png,Basic,None,None,70,SM198,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped
8445,Bulbasaur,det1-1,1.0,https://images.pokemontcg.io/det1/1.png,https://images.pokemontcg.io/det1/1_hires.png,Basic,None,None,60,1,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped
9387,Bulbasaur,base4-67,1.0,https://images.pokemontcg.io/base4/67.png,https://images.pokemontcg.io/base4/67_hires.png,Basic,None,None,40,67,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped
619,Bulbasaur,ex6-55,1.0,https://images.pokemontcg.io/ex6/55.png,https://images.pokemontcg.io/ex6/55_hires.png,Basic,None,None,50,55,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped
4465,Bulbasaur,ex14-46,1.0,https://images.pokemontcg.io/ex14/46.png,https://images.pokemontcg.io/ex14/46_hires.png,Basic,None,None,50,46,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped
8629,Bulbasaur,ex4-39,1.0,https://images.pokemontcg.io/ex4/39.png,https://images.pokemontcg.io/ex4/39_hires.png,Basic,None,None,50,39,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped
4800,Bulbasaur,bw5-1,1.0,https://images.pokemontcg.io/bw5/1.png,https://images.pokemontcg.io/bw5/1_hires.png,Basic,None,None,60,1,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped
6809,Bulbasaur,pop2-12,1.0,https://images.pokemontcg.io/pop2/12.png,https://images.pokemontcg.io/pop2/12_hires.png,Basic,None,None,40,12,...,49.0,49.0,45.0,poison,grass,70.0,green,plant,generation-i,quadruped


In [59]:
unique_pokemon = bulbapedia['pokemon_name'].to_list()

for x in unique_pokemon:
    pokemon_card_data = merged_data.loc[merged_data['pokemon_name']== x , ['card_name', 'card_id', 'national_pokedex_number', 'image_url',
       'image_url_hi_res', 'card_subtype', 'card_ability',
       'card_ancient_trait', 'card_hp', 'card_number', 'card_artist',
       'card_rarity', 'card_series', 'card_set', 'card_set_code',
       'card_converted_retreat_cost', 'card_text', 'card_types',
       'card_attacks', 'card_weaknesses', 'card_resistances',
       'card_evolves_from']]
    post = {
        'pokemon_name': x,
        'card_data': {
            'card_name': pokemon_card_data['card_name'].to_list(),
            'card_id': pokemon_card_data['card_id'].to_list(),
            'national_pokedex_number': pokemon_card_data['national_pokedex_number'].to_list(), 
            'image_url': pokemon_card_data['image_url'].to_list(),
            'image_url_hi_res': pokemon_card_data['image_url_hi_res'].to_list(), 
            'card_subtype': pokemon_card_data['card_subtype'].to_list(), 
            'card_ability': pokemon_card_data['card_ability'].to_list(),
            'card_ancient_trait': pokemon_card_data['card_ancient_trait'].to_list(), 
            'card_hp': pokemon_card_data['card_hp'].to_list(), 
            'card_number': pokemon_card_data['card_number'].to_list(), 
            'card_artist': pokemon_card_data['card_artist'].to_list(),
            'card_rarity': pokemon_card_data['card_rarity'].to_list(), 
            'card_series': pokemon_card_data['card_series'].to_list(), 
            'card_set': pokemon_card_data['card_set'].to_list(), 
            'card_set_code': pokemon_card_data['card_set_code'].to_list(),
            'card_converted_retreat_cost': pokemon_card_data['card_converted_retreat_cost'].to_list(), 
            'card_text': pokemon_card_data['card_text'].to_list(), 
            'card_types': pokemon_card_data['card_types'].to_list(),
            'card_attacks': pokemon_card_data['card_attacks'].to_list(), 
            'card_weaknesses': pokemon_card_data['card_weaknesses'].to_list(), 
            'card_resistances': pokemon_card_data['card_resistances'].to_list(),
            'card_evolves_from': pokemon_card_data['card_evolves_from'].to_list()
        },
        'national_pokedex_number':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['national_pokedex_number'].item(), 
        'sprite':  bulbapedia.loc[bulbapedia['pokemon_name']== x][ 'sprite'].item(), 
        'pokemon_name':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['pokemon_name'].item(), 
        'pokemon_weight':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['pokemon_weight'].item(),
        'pokemon_height':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['pokemon_height'].item(), 
        'base_stats':{'speed':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['speed'].item(), 
                      'special_defense':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['special_defense'].item(), 
                      'special_attack':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['special_attack'].item(),
                      'defense':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['defense'].item(), 
                      'attack':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['attack'].item(), 
                      'hp':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['hp'].item(),},
        'types': {'pokemon_type1':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['pokemon_type1'].item(), 
                  'pokemon_type2':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['pokemon_type2'].item(),},
        'base_hapiness':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['base_hapiness'].item(), 
        'color_group':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['color_group'].item(), 
        'egg_group':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['egg_group'].item(), 
        'generation':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['generation'].item(), 
        'shape':  bulbapedia.loc[bulbapedia['pokemon_name']== x]['shape'].item()
    }
    client.pokemon_db.test_pokemon.insert_one(post)

In [58]:
post

{'pokemon_name': 'zeraora',
 'card_data': {'card_name': ['Zeraora',
   'Zeraora-GX',
   'Zeraora',
   'Zeraora-GX',
   'Zeraora-GX'],
  'card_id': ['sm10-60', 'sm8-221', 'sm9-52', 'sm8-86', 'sm8-201'],
  'national_pokedex_number': [807.0, 807.0, 807.0, 807.0, 807.0],
  'image_url': ['https://images.pokemontcg.io/sm10/60.png',
   'https://images.pokemontcg.io/sm8/221.png',
   'https://images.pokemontcg.io/sm9/52.png',
   'https://images.pokemontcg.io/sm8/86.png',
   'https://images.pokemontcg.io/sm8/201.png'],
  'image_url_hi_res': ['https://images.pokemontcg.io/sm10/60_hires.png',
   'https://images.pokemontcg.io/sm8/221_hires.png',
   'https://images.pokemontcg.io/sm9/52_hires.png',
   'https://images.pokemontcg.io/sm8/86_hires.png',
   'https://images.pokemontcg.io/sm8/201_hires.png'],
  'card_subtype': ['Basic', 'GX', 'Basic', 'GX', 'GX'],
  'card_ability': [None,
   {'name': 'Thunderclap Zone',
    'text': 'Each of your Pokémon that has any Lightning Energy attached to it has no Re

In [47]:
bulbapedia.loc[merged_data['pokemon_name']== 'bulbasaur']['pokemon_weight'].item()

253.0

In [31]:
bulbapedia.columns

Index(['national_pokedex_number', 'sprite', 'pokemon_name', 'pokemon_weight',
       'pokemon_height', 'speed', 'special_defense', 'special_attack',
       'defense', 'attack', 'hp', 'pokemon_type1', 'pokemon_type2',
       'base_hapiness', 'color_group', 'egg_group', 'generation', 'shape'],
      dtype='object')

In [50]:
 bulbapedia.loc[merged_data['pokemon_name']== x]['national_pokedex_number']

33     3  
324    373
460    556
192    718
Name: national_pokedex_number, dtype: int64

In [52]:
 bulbapedia.loc[bulbapedia['pokemon_name']== x]

,national_pokedex_number,sprite,pokemon_name,pokemon_weight,pokemon_height,speed,special_defense,special_attack,defense,attack,hp,pokemon_type1,pokemon_type2,base_hapiness,color_group,egg_group,generation,shape
33,3,https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/3.png,venusaur,1000.0,20.0,80.0,100.0,100.0,83.0,82.0,80.0,poison,grass,70.0,green,plant,generation-i,quadruped
